In [165]:
# coding=utf-8
import os
import re
import csv
import time
import xlrd
import shutil
# 记录所有数据
data_all = {}
# 时长不足
data_no_time = []
# 没签到  
data_no_sign_time = []
# 名字格式错误ALL
data_name_error_all = []
# 名字格式错误
data_name_error_one = []
# 保存头部
csv_headers = [ '姓名', '观看次数']
csv_one_headers = ['姓名','观看直播时长','签到','回放时长','直播时间']

In [166]:
class Utils(object):
    """
    工具类
    """
    def select_number(self, text):
        """
        清洗出数字
        :text:文本
        """
        number_tmp = re.search(r'\d+', text) # 避免格式错误导致NoneType
        number = '' # 学号
        if number_tmp:
            number = number_tmp.group()
        return number
    def select_no_number(self, text):
        """
        清除数字的文本
        :text:文本
        """
        txt = re.sub('[^A-Za-z\u4e00-\u9fa5]', '', text).lstrip()
        return txt
    def select_bool(self, text):
        """
        识别bool类型
        :text:文本
        """
        txt = re.search(r'是|否',text)
        if txt:
            txt = txt.group()
        else: # 老师没签到
            txt = '是'
        return txt
    def clean_time(self, resource_text, dist_text):
        """
        清洗时长
        :text: 时长
        """
        txt = resource_text
        if not resource_text:
            return '不足一分钟'
        return txt

In [167]:
class Student(object):
    """
    数据处理类
    """
    def clean_one(self, excel):
        """
        处理单个excel
        :excel:文件路径
        """
        # 单个文本数据
        data_one_all = {}
        # 打开文件
        data = xlrd.open_workbook(excel)
        data_sheet = data.sheet_by_name("数据导出")
        all_row = data_sheet.nrows  # 总行数
        utils = Utils()
        all_time = utils.select_number(data_sheet.row_values(2)[1]) # 直播真实时长
        view_time =  ' '.join(data_sheet.row_values(2)[0].split(' ')[:2]) # 直播时间
        
        is_sign = False # 3月份没签到功能
        if len(data_sheet.row_values(5)) > 10:
            is_sign = True
        else:
            print('这节课还没签到功能：',view_time)
            
        for i in range(5, all_row):
            row_date = data_sheet.row_values(i)     
            name = utils.select_no_number(row_date[3]) # 姓名
            time = utils.select_number(row_date[7]) # 直播时长
            time = utils.clean_time(time, all_time) # 清洗后的时长
            old_time = utils.select_number(row_date[9]) # 回放时长
            old_time = utils.clean_time(old_time, all_time) # 清洗后的回放时长
            
            if is_sign:
                sign = utils.select_bool(row_date[10]) # 是否签到
            else:
                sign = '是'
                
            if data_one_all.setdefault(name,0):
                time_tmp = 0
                if data_one_all[name]['观看直播时长'] != '不足一分钟':
                    time_tmp += int(data_one_all[name]['观看直播时长'])    
                if time != '不足一分钟':
                    time_tmp += int(time)
                data_one_all[name]['观看直播时长'] = str(time_tmp)
                if sign == '是' and data_one_all[name]['签到'] == '否':
                    data_one_all[name]['签到'] = '是'
            else:
                data_one_all[name] = {
#                     '学号': number,
                    '姓名': name,
                    '观看直播时长': time,
                    '签到' : sign,
                    '回放时长':old_time,
                    '直播时间': view_time
                }
        return data_one_all , all_time
    def stat_data(self, data, all_time):
        """
        统计所有excel文件
        :all_data:单个清洗后的数据
        :all_time:当次直播时长
        """
        for one_data in data:
            time_tmp = False 
            if data[one_data]['观看直播时长'] != '不足一分钟' and data[one_data]['回放时长'] != '不足一分钟':
#                 time_tmp = int(data[one_data]['观看直播时长']) > 60 
                # 控制时长筛选条件
                if int(data[one_data]['观看直播时长']) > 60:
                    time_tmp = True
                else:
                    if int(data[one_data]['回放时长']) > 60:
                        time_tmp = True
            sign_tmp = data[one_data]['签到'] == '是'
#             if not (sign_tmp and time_tmp):
            if not time_tmp:
                data_no_sign_time.append(data[one_data]) 
            if data_all.setdefault(one_data, 0):
                data_all[data[one_data]['姓名']]['观看次数'] += time_tmp # 观看 + 1 
#                 data_all[data[one_data]['姓名']]['签到次数'] += sign_tmp # 签到 + 1
            else:
                """
                "姓名" ：{
                    "姓名" ： "xxx",
                    "直播总次数": "xx",
                }
                """
                data_all[data[one_data]['姓名']] = {
#                     '学号': data[one_data]['学号'],
                    '姓名': data[one_data]['姓名'],
                    '观看次数': int(time_tmp),
#                     '签到次数': int(sign_tmp)
                }
    def clean_error(self,data,path):
        """
        清洗学号格式错误人员
        :data:清洗的数据
        @return num 总人数
        """
        path = re.search(r'["信息"|"教技"|"土木"]+',path)
        if path:
            path = path.group()
        num = 0
        for data_tmp in list(data):
            if isinstance(data,dict):
                data_tmp_title = re.search(r'["信息"|"教技"|"土木"]+',data_tmp)
                if not data_tmp_title or data_tmp_title.group() != path:
                    data_name_error_all.append(data[data_tmp])
                    data.pop(data_tmp)
                    continue
            else:
                data_tmp_title = re.search(r'["信息"|"教技"|"土木"]+',data_tmp['姓名'])
                if not data_tmp_title or data_tmp_title.group() != path:
                    data_name_error_one.append(data_tmp)
                    data.remove(data_tmp)
                    continue
            num += 1
        return num

In [168]:
class Save(object):
    
    """
    数据存储类
    """
    def csv_save(self, data, csv_name, headers):
        """
        csv存储数据
        :data:存储的数据
        :文件名字
        """
        utils = Utils()
        if isinstance(data,dict): # dict转list
            data = [i for i in data.values()]
        with open(csv_name, mode = 'a', newline='') as f:
            writer = csv.DictWriter(f, headers)
            for row in data:
                writer.writerow(row)

In [173]:
class Cluster(object):
    """
    聚类分析
    """
    def cluster_data(self, data):
        """
        汇聚数据
        :data:数据
        """
        data_tmp = {}
        for tmp in data:
            data_tmp.setdefault(tmp['姓名'],[])
            data_tmp[tmp['姓名']].append(tmp)
        # 整理格式
        return self.cluster_format(data_tmp)
    def cluster_format(self, data):
        """
        格式整理
        :data:数据
        """
        data_tmp = []
        for (key, value) in data.items():
            for tmp in value:
                data_tmp.append(tmp)
        return data_tmp

In [174]:
if __name__ == '__main__':
    student = Student()
    for path in os.listdir('data'):
        if os.path.exists(path): # 防止每次追加
            shutil.rmtree(path)
        os.makedirs(path)
        # 每次更新,防止追加
        data_all= {}
        data_no_sign_time = []
        data_name_error_all = []
        data_name_error_one = []
        with open('{}/data.csv'.format(path),mode = 'a', newline='') as f:
                writer = csv.DictWriter(f, csv_headers)
                writer.writeheader()
        with open('{}/data_name_error_all.csv'.format(path),mode = 'a', newline='') as f:
                writer = csv.DictWriter(f, csv_headers)
                writer.writeheader()
        # 写入时长未到&没签到
        with open('{}/data_error.csv'.format(path),mode = 'a', newline='') as f:
                writer = csv.DictWriter(f, csv_one_headers)
                writer.writeheader()
        with open('{}/data_name_error_one.csv'.format(path),mode = 'a', newline='') as f:
                writer = csv.DictWriter(f, csv_one_headers)
                writer.writeheader()
        for path_child in os.listdir('data/{}'.format(path)):
            path_tmp = 'data/{}/{}'.format(path,path_child)
            data , all_time = student.clean_one(path_tmp) # 单个excel文件数据ss
            student.stat_data(data,all_time) # 所有数据
        cluster = Cluster()
        data_no_sign_time = cluster.cluster_data(data_no_sign_time)
        num = student.clean_error(data_all,path) # 清洗data_all学号格式错误的人员
        num_no_sign_time = student.clean_error(data_no_sign_time, path) # 清洗data_no_time姓名格式错误的人员s
        # 存储全部数据
        save = Save()
        save.csv_save(data_all, '{}/data.csv'.format(path),  csv_headers)
        save.csv_save(data_name_error_all, '{}/data_name_error_all.csv'.format(path),  csv_headers)
        save.csv_save(data_no_sign_time,'{}/data_error.csv'.format(path),csv_one_headers) # 记录
        save.csv_save(data_name_error_one, '{}/data_name_error_one.csv'.format(path),  csv_one_headers)